In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgbm
#from xgboost import XGBRegressor
#import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import mplcyberpunk
import seaborn as sns

In [2]:
train_time = 1 * 60 * 60
objective = 'lgbmregressor'

In [3]:
sys.path.append(os.path.realpath('..'))

In [4]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [5]:
#features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
#       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
#       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

features = ['V1', 'V2', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
target = ['Class']

In [6]:
np.unique(train_df[target], return_counts=True)

(array([0, 1], dtype=int64), array([218660,    469], dtype=int64))

In [7]:
train_df[features] = train_df[features].apply(lambda iterator: ((iterator.max() - iterator)/(iterator.max() - iterator.min())).round(2))
test_df[features] = test_df[features].apply(lambda iterator: ((iterator.max() - iterator)/(iterator.max() - iterator.min())).round(2))

In [8]:
np.unique(train_df[target], return_counts=True)

(array([0, 1], dtype=int64), array([218660,    469], dtype=int64))

In [9]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [10]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}
train_df_score_keeper = train_df.copy()

train_df_score_keeper['record_weight'] = 0

class callbacker():
    def __init__(self):
        self.totals = []
        
    def add_totals(self, payload):
        #print(payload.evaluation_result_list[0][2])
        self.totals.append(payload.evaluation_result_list[0][2])
        
    def get_totals(self):
        return self.totals
    
    def get_mean(self):
        return sum(np.array(self.totals)) / len(self.totals)
    
best_auc = 0

def objective_v3(trial):
    global best_auc
    oof = np.zeros(len(X))
    scores = []
    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'max_depth': trial.suggest_int('max_depth', 3, 8), 
        #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 100),
        #'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        #'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 1),
        #'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 1),
        #'subsample': trial.suggest_float('subsample', 0.1, 1),
        #'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.1, 5),
        #'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1),
        #'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1),
        #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        #'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 5), 
        #'n_estimators': trial.suggest_int('n_estimators', 10, 100000),
        'objective':'binary',
    } 
    reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for z in range(10):
        for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
            cber = callbacker()
            print(10*"=", f"Fold={fold+1}", 100*"=")
            X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
            y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 

            reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True), cber.add_totals])

            train_df_score_keeper.loc[train_idx, 'record_weight'] = (train_df_score_keeper.loc[train_idx, 'record_weight'] + cber.get_mean()) / 2

            oof[valid_idx] = reg.predict(X_valid)

        roc_auc = roc_auc_score(Y, oof)

        if roc_auc > best_auc:
            best_auc = roc_auc
            train_final_preds = reg.predict(train_df[features])
            test_final_preds = reg.predict(test_df[features])
            train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
            train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)
            test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
            test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)
        
    
    param_grid_history[roc_auc] = param_grid
    train_df_score_keeper.to_csv('1000kfoldwithweights.csv')

    return roc_auc

In [11]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v3, n_trials=1000)

[I 2023-01-29 12:49:36,531] A new study created in memory with name: lgbmregressor


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[9]	valid_0's auc: 0.729357
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[71]	valid_0's auc: 0.686086
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[16]	valid_0's auc: 0.690211
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is

[I 2023-01-29 12:50:03,461] Trial 0 finished with value: 0.6986228778231901 and parameters: {'learning_rate': 0.8757216287451924, 'max_depth': 3, 'reg_lambda': 2}. Best is trial 0 with value: 0.6986228778231901.


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[2]	valid_0's auc: 0.75365
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1]	valid_0's auc: 0.737307
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[5]	valid_0's auc: 0.719335
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[

[I 2023-01-29 12:50:40,630] Trial 1 finished with value: 0.7345414413084388 and parameters: {'learning_rate': 0.726908655007202, 'max_depth': 8, 'reg_lambda': 5}. Best is trial 1 with value: 0.7345414413084388.


...

[I 2023-01-29 13:34:40,303] Trial 84 finished with value: 0.7927392021611767 and parameters: {'learning_rate': 0.37367243648650694, 'max_depth': 4, 'reg_lambda': 5}. Best is trial 71 with value: 0.8106495231568438.


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[18]	valid_0's auc: 0.819303
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[20]	valid_0's auc: 0.803963
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[27]	valid_0's auc: 0.76732
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is

[I 2023-01-29 13:35:12,655] Trial 85 finished with value: 0.8046467805359139 and parameters: {'learning_rate': 0.17215852128114958, 'max_depth': 6, 'reg_lambda': 3}. Best is trial 71 with value: 0.8106495231568438.


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[4]	valid_0's auc: 0.78479
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[3]	valid_0's auc: 0.775457
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[8]	valid_0's auc: 0.746807
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[

[W 2023-01-29 13:35:33,140] Trial 86 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_10896\2229885829.py", line 55, in objective_v3
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True), cber.add_totals])
  File "D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py", line 895, in fit
    super().fit(X, y, sample_weight=sample_weight, init_score=init_score,
  File "D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster = train(
  File "D:\source\repos\venv\Python310\lib\site-packages\lightgbm\engine.py", line 299, in train
    evaluation_result_list.extend(booster.eval_valid(feval))
  File "D:\source\repos\venv\Python310\lib\sit

In [ ]:
train_df_score_keeper['record_weight'].describe()

In [ ]:
train_df_score_keeper['record_weight'][-200:-100]

In [ ]:
train_df_score_keeper.to_csv('1000kfoldwithweights.csv')

In [ ]:
#train_df = train_df_score_keeper[train_df_score_keeper['record_weight'] > train_df_score_keeper['record_weight'].describe()['25%']].copy()
#train_df.reset_index(inplace=True)

In [ ]:
train_df

In [ ]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [ ]:
def train(param_grid):
    reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.20)
    
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)
    
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-1:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

In [ ]:
plt.style.use("cyberpunk")

histplot_hyperparams = {
        'alpha':0.4,
        'stat':'percent',
        'bins':100
    }

n_cols = 1
n_rows = 1
cell_with_dim = 6
cell_height_dim = 3

fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))

sns.histplot(train_final_preds, ax=ax, **histplot_hyperparams)
#sns.histplot(train_df[target], ax=ax, **histplot_hyperparams)

mplcyberpunk.make_lines_glow(ax)
print(np.unique(train_final_preds, return_counts=True))
print(roc_auc_score(train_df[target], train_final_preds))
plt.show()

In [ ]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)